In [ ]:
%%javascript
$('#run_all_cells_below').click()


# Property Based Testing
## (Using Hypothesis)<br><br><br>
### Amsterdam Python Meetup
### 27 April 2017<br><br><br>
### Daniel Bradburn


Property based testing

Choosing properties

Generating data

Model based testing

Django

Examples

say we have a run length encoding function. We encode a string as characters and the number of consecutive occurrences of that character. let's just test this out with something simple

In [ ]:
def encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
encode('hello')

and we also have a decode function which reconstructs the string let's just check this function, let's use the output from the encode

In [ ]:
def decode(lst):
    return ''.join(c * n for c, n in lst)

In [ ]:
decode([('h', 1), ('e', 1), ('l', 2), ('o', 1)])

but it's probably best to formalize this in a unit test. I'm using pytest here, but you could use unittest or your favourite test runner, the principal is the same.

In [ ]:
def test_run_length_encode():
    input_data = "hello"
    expected = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    actual = encode(input_data)
    assert actual == expected

In [ ]:
!py.test -k test_run_length_encode -q

In [ ]:
def test_run_length_decode():
    input_data = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    expected = "hello"
    actual = decode(input_data)
    assert actual == expected

In [ ]:
!py.test -k test_run_length_decode -q

In [ ]:
import pytest

examples = ['hello', 'python', 'uhm...']

@pytest.mark.parametrize('input_data', examples)
def test_parameterized_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_parameterized_run_length_encode_decode -q

In [ ]:
import random, string

random.seed(0)

random_letter = lambda: random.choice(string.ascii_letters)
random_range = lambda m: range(random.randint(0, m))
random_word = lambda m: (random_letter() for i in random_range(m))
random_words = lambda n, m: (''.join(random_word(m)) for n in range(n))

@pytest.mark.parametrize('input_data', random_words(5, 10))
def test_fuzzed_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_fuzzed_run_length_encode_decode -q

In [ ]:
from hypothesis import strategies as st
from hypothesis import given

@given(st.text())
def test_property_based_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_property_based_run_length_encode_decode -q

In [ ]:
def encode_fixed(input_string):
    count = 1
    prev = ''
    lst = []
    character = ''
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
@given(st.text())
def test_property_based_run_length_encode_fixed_decode(input_data):
    assert decode(encode_fixed(input_data)) == input_data

In [ ]:
!py.test -k test_property_based_run_length_encode_fixed_decode -q

In [ ]:
class Queue(object):

    def __init__(self, max_size):
        self._buffer = [None] * max_size
        self._in, self._out, self.max_size = 0, 0, max_size

    def put(self, item):
        self._buffer[self._in] = item
        self._in = (self._in + 1) % self.max_size

    def get(self):
        result = self._buffer[self._out]
        self._out = (self._out + 1) % self.max_size
        return result

    def __len__(self):
        return (self._in - self._out) % self.max_size

In [ ]:
from hypothesis import strategies as st
from hypothesis.stateful import RuleBasedStateMachine, rule, precondition

class QueueMachine(RuleBasedStateMachine):

    SystemUnderTest, Model = Queue, list
    system_under_test, model, max_size = None, None, 0

    @precondition(lambda self: self.system_under_test is None)
    @rule(max_size=st.integers(min_value=1, max_value=10))
    def new(self, max_size):
        self.system_under_test = self.SystemUnderTest(max_size)
        self.model = self.Model()
        self.max_size = max_size

    @precondition(lambda self: self.system_under_test is not None)
    @rule(item=st.integers())
    def put(self, item):
        self.system_under_test.put(item)
        self.model.append(item)

    @precondition(lambda self: self.system_under_test is not None
                               and len(self.model))
    @rule()
    def get(self):
        actual = self.system_under_test.get()
        model = self.model.pop()
        assert actual == model

    @precondition(lambda self: self.system_under_test is not None)
    @rule()
    def size(self):
        actual = len(self.system_under_test)
        model = len(self.model)
        assert actual == model

In [ ]:
test_model_based_1 = QueueMachine.TestCase

In [ ]:
!py.test -k test_model_based_1 -q --tb short

In [ ]:
class QueueMachine2(QueueMachine):

    @precondition(lambda self: self.system_under_test is not None)
    @rule(item=st.integers())
    def put(self, item):
        self.system_under_test.put(item)
        self.model.insert(0, item)

In [ ]:
test_model_based_2 = QueueMachine2.TestCase

In [ ]:
!py.test -k test_model_based_2 -q --tb short

In [ ]:
class QueueMachine3(QueueMachine2):

    @precondition(lambda self: self.system_under_test is not None
                               and len(self.model) < self.max_size)
    @rule(item=st.integers())
    def put(self, item):
        super(QueueMachine3, self).put(item)

In [ ]:
test_model_based_3 = QueueMachine3.TestCase

In [ ]:
!py.test -k test_model_based_3 -q --tb short

In [ ]:
class Queue2(Queue):
    def __init__(self, max_size):
        super(Queue2, self).__init__(max_size + 1)

class QueueMachine4(QueueMachine3):
    SystemUnderTest = Queue2

test_model_based_4 = QueueMachine4.TestCase

In [ ]:
!py.test -k test_model_based_4 -q --tb short

In [ ]:
%%javascript
$('#clear_all_output').click()


In [ ]:
%%HTML
<link href="https://fonts.googleapis.com/css?family=Poppins" rel="stylesheet">
<style>body { font-family: 'Poppins', serif !important; }</style>
